## Julia code for simulation

In [1]:
## necessary packages

using Distributions
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
using Random
using SpecialFunctions # Matern functions
using MLBase         #cross-validation  

In [122]:
include("./utils.jl")

stacking_prediction_LSE (generic function with 2 methods)

In [3]:
# Set the parameters of the simulated data #
p = 2;      # No. covariates
β = [1.0 2.0]; #regression coeff
σ2 = 1.0; ϕ = 7.0; ν = 1.0; τ2 = 1.0; # hyperparmeters in matern

In [4]:
## Generate simulation data ##
Random.seed!(1);
N = 400;                     # No. all positions
N_ho = 100;                  # No. held out positions
ind_mod = 1:(N - N_ho);      # index of training observations
coords = rand(2, N);         # random location over unit square (2 by N)
X = vcat(fill(1.0, (1, N)), rand(Normal(), (1, N)));          # design matrix (p by N)
D = pairwise(Euclidean(), coords, dims = 2);                  # distance matrix
Cov = Symmetric(Maternlu.(UpperTriangular(D), 
        ν = ν, ϕ = ϕ, σ2 = σ2))                               # covariance matrix
z = rand(MvNormal(Cov), 1);                                   # latent process
y = (β * X)[1,:] + z[:,1] + sqrt(τ2) * rand(Normal(), N);     # response

In [5]:
using BenchmarkTools

In [6]:
## candidate values of hyperparameters for stacking ##
deltasq_grid = [0.1, 0.5, 1, 2];
phi_grid = [3, 9, 15, 21];
nu_grid = [0.5, 1, 1.5, 1.75];

In [7]:
## priors parameters ##

μβ = fill(0.0, p); inv_V_β = Diagonal(ones(p) * 0.25); # set Vr^{-1} be zero for the simulation...
aσ = 2.0; bσ = 2.0;

In [8]:
label = "LSE"; #stacking of means
J = 300;

In [9]:
label = "LP";  #stacking of predictive densities
J = 300;       # sample size for computing posterior expectation

In [90]:
# pre-computation and pre-allocation #
K_fold = 10;
N = size(X, 2);
CV_ind_ls = collect(Kfold(N, K_fold)); # index of train data in CV
CV_ind_hold_ls = [setdiff(1:N, CV_ind_ls[k]) for k in 1:K_fold]; # index of held-out data in CV
N_grid = length(deltasq_grid) * length(phi_grid) * length(nu_grid);
nk_list = [length(x) for x in CV_ind_ls]; # be careful, different from the nk_list in my R code
nk_k_list = [(N - x) for x in nk_list];   # This is the nk_list in my R code


if X == Nothing()
    p = 0;
else
    p = size(X, 1);
    inv_V_μ_β = inv_V_β * μβ;
    XTX = X * X'; XTy = X * y;
    XTX_list = [XTX - X[:, CV_ind_hold_ls[k]] * X[:, CV_ind_hold_ls[k]]' for k in 1:K_fold];
    XTy_list = [XTy - X[:, CV_ind_hold_ls[k]] * y[CV_ind_hold_ls[k]] for k in 1:K_fold];
end

if label == "LSE"
    y_expect = Array{Float64, 2}(undef, N, N_grid);
elseif label == "LP"
    lp_expect = Array{Float64, 2}(undef, N, N_grid);
    y_sq_sum_list = [norm(y[CV_ind_ls[k]])^2 for k in 1:K_fold];
else 
    print("label has to be LSE or LP");
end

grid_phi_nu = vcat([[x y] for x in phi_grid, y in nu_grid]...);
grid_all = vcat([[x y z] for x in phi_grid, y in nu_grid, z in deltasq_grid]...);
L_grid_deltasq  = length(deltasq_grid);

In [91]:
k = 1;
i1 = 1;
i2 = 1;
phi_pick = grid_phi_nu[i1, 1];
nu_pick = grid_phi_nu[i1, 2];
deltasq_pick = deltasq_grid[i2];

In [375]:
invR_nk = inv(cholesky(Symmetric(Maternlu.(
        UpperTriangular(pairwise(Euclidean(), coords[:, CV_ind_ls[k]], dims = 2)), 
        ν = nu_pick, ϕ = phi_pick))));
R_k_nk = Maternlu.(pairwise(Euclidean(), coords[:, CV_ind_hold_ls[k]], 
        coords[:, CV_ind_ls[k]], dims = 2), ν = nu_pick, ϕ = phi_pick);
if p == 0
    chol_inv_M = cholesky(invR_nk + Diagonal(repeat([1 / deltasq_pick], nk_list[k]))); 
    u = y[CV_ind_ls[k]] /  deltasq_pick;
else
    chol_inv_M = cholesky(Symmetric(vcat(
                hcat(XTX_list[k] / deltasq_pick + inv_V_β, X[:, CV_ind_ls[k]] / deltasq_pick),
                hcat(zeros(nk_list[k], p), 
                    invR_nk + Diagonal(repeat([1 / deltasq_pick], nk_list[k])))), :U));
    u = [inv_V_μ_β + XTy_list[k] / deltasq_pick; y[CV_ind_ls[k]] /  deltasq_pick];
end

ldiv!(chol_inv_M.U', u);  # u2 = chol_inv_M.L \ u;

if label == "LSE"
    ## Stacking based on expectation
    # compute expectation of response in fold k
    ldiv!(chol_inv_M.U, u); # compute the posterior expectation of β and z u3 = chol_inv_M.L' \ u2;

    if p == 0
        #z_U_expect = R_k_nk * (invR_nk * u);
        y_expect[CV_ind_hold_ls[k], (i1 - 1) * L_grid_deltasq + i2] = 
            (R_k_nk * invR_nk) * u;
    else
        #z_U_expect = R_k_nk * (invR_nk * u[(p + 1):end]);
        y_expect[CV_ind_hold_ls[k], (i1 - 1) * L_grid_deltasq + i2] = 
            (X[:, CV_ind_hold_ls[k]]' * u[1:p]) + (R_k_nk * invR_nk) * u[(p + 1):end];
    end
else
    ## Stacking based on log point-wise predictive density
    if p == 0
        b_star = bσ + 0.5 * (y_sq_sum_list[k] / deltasq_pick - norm(u)^2);
    else
        b_star = bσ + 0.5 * (y_sq_sum_list[k] / deltasq_pick + 
            dot(μβ, inv_V_μ_β) - norm(u)^2);
    end
    a_star = aσ + nk_list[k] / 2;

    ## generate posterior samples ##
    σ2_sam = rand(InverseGamma(a_star, b_star), J);

    ## compute the expected response on unobserved locations ##
    γ_sam = rand(Normal(), (J, (nk_list[k] + p)));   # each row is a sample
    lmul!(Diagonal(sqrt.(σ2_sam)), γ_sam);
    add_to_row!(γ_sam, u);          
    rdiv!(γ_sam, chol_inv_M.U');            # γ_sam' = backsolve(chol_inv_M, gamma.sam)

    # M_r the matrix of log ratios (lp), first, store the expected responses on the held out fold
    if p == 0
        M_r = γ_sam * (invR_nk * R_k_nk');
    else
        M_r = γ_sam[:, (1:p)] * X[:, CV_ind_hold_ls[k]] + 
              γ_sam[:, (p+1):end] * (invR_nk * R_k_nk');
    end

    # the matrix of log ratios (lp)
    minus_to_row!(M_r, y[CV_ind_hold_ls[k]]);
    lmul!(Diagonal(sqrt.(1 / (deltasq_pick * σ2_sam))), M_r);
    square_and_plus!(M_r, log(2*pi));
    add_to_column!(M_r, log.(deltasq_pick * σ2_sam));
    M_r .*= -0.5;
    lp_expect[CV_ind_hold_ls[k], (i1 - 1) * L_grid_deltasq + i2] = log.(mean(exp.(M_r), dims = 1))[1,:];
end  
0.0

0.0

In [44]:
prog = Progress(size(grid_phi_nu, 1), 1, "Computing initial pass...", 50)
for i1 in 1:size(grid_phi_nu, 1)
    phi_pick = grid_phi_nu[i1, 1];
    nu_pick = grid_phi_nu[i1, 2];
    for k in 1:K_fold
        invR_nk = inv(cholesky(Symmetric(Maternlu.(
                UpperTriangular(pairwise(Euclidean(), coords[:, CV_ind_ls[k]], dims = 2)), 
                ν = nu_pick, ϕ = phi_pick))));
        R_k_nk = Maternlu.(pairwise(Euclidean(), coords[:, CV_ind_hold_ls[k]], 
                coords[:, CV_ind_ls[k]], dims = 2), ν = nu_pick, ϕ = phi_pick);
        for i2 in 1:L_grid_deltasq
            deltasq_pick = deltasq_grid[i2];
            if p == 0
                chol_inv_M = cholesky(invR_nk + Diagonal(repeat([1 / deltasq_pick], nk_list[k]))); 
                u = y[CV_ind_ls[k]] /  deltasq_pick;
            else
                chol_inv_M = cholesky(Symmetric(vcat(
                            hcat(XTX_list[k] / deltasq_pick + inv_V_β, X[:, CV_ind_ls[k]] / deltasq_pick),
                            hcat(zeros(nk_list[k], p), 
                                invR_nk + Diagonal(repeat([1 / deltasq_pick], nk_list[k])))), :U));
                u = [inv_V_μ_β + XTy_list[k] / deltasq_pick; y[CV_ind_ls[k]] /  deltasq_pick];
            end
            
            ldiv!(chol_inv_M.U', u);  # u2 = chol_inv_M.L \ u;
            
            if label == "LSE"
                ## Stacking based on expectation
                # compute expectation of response in fold k
                ldiv!(chol_inv_M.U, u); # compute the posterior expectation of β and z u3 = chol_inv_M.L' \ u2;
                
                if p == 0
                    y_expect[CV_ind_hold_ls[k], (i1 - 1) * L_grid_deltasq + i2] = 
                        (R_k_nk * invR_nk) * u;
                else
                    y_expect[CV_ind_hold_ls[k], (i1 - 1) * L_grid_deltasq + i2] = 
                        (X[:, CV_ind_hold_ls[k]]' * u[1:p]) + (R_k_nk * invR_nk) * u[(p + 1):end];
                end
            else
                ## Stacking based on log point-wise predictive density
                if p == 0
                    b_star = bσ + 0.5 * (y_sq_sum_list[k] / deltasq_pick - norm(u)^2);
                else
                    b_star = bσ + 0.5 * (y_sq_sum_list[k] / deltasq_pick + 
                        dot(μβ, inv_V_μ_β) - norm(u)^2);
                end
                a_star = aσ + nk_list[k] / 2;
                
                ## generate posterior samples ##
                σ2_sam = rand(InverseGamma(a_star, b_star), J);
                
                ## compute the expected response on unobserved locations ##
                γ_sam = rand(Normal(), (J, (nk_list[k] + p)));   # each row is a sample
                lmul!(Diagonal(sqrt.(σ2_sam)), γ_sam);
                add_to_row!(γ_sam, u);          
                rdiv!(γ_sam, chol_inv_M.U');            # γ_sam' = backsolve(chol_inv_M, gamma.sam)
                
                # M_r the matrix of log ratios (lp), first, store the expected responses on the held out fold
                if p == 0
                    M_r = γ_sam * (invR_nk * R_k_nk');
                else
                    M_r = γ_sam[:, (1:p)] * X[:, CV_ind_hold_ls[k]] + 
                          γ_sam[:, (p+1):end] * (invR_nk * R_k_nk');
                end
                
                # the matrix of log ratios (lp)
                minus_to_row!(M_r, y[CV_ind_hold_ls[k]]);
                lmul!(Diagonal(sqrt.(1 / (deltasq_pick * σ2_sam))), M_r);
                square_and_plus!(M_r, log(2*pi));
                add_to_column!(M_r, log.(deltasq_pick * σ2_sam));
                M_r .*= -0.5;
                lp_expect[CV_ind_hold_ls[k], (i1 - 1) * L_grid_deltasq + i2] = log.(mean(exp.(M_r), dims = 1))[1,:];
            end          
        end
    end
    next!(prog)
end

Computing initial pass...100%|██████████████████████████████████████████████████| Time: 0:00:06


In [92]:
function stacking_prediction(coords, nu_pick, phi_pick, deltasq_grid, 
        L_grid_deltasq, k, CV_ind_ls, CV_ind_hold_ls, p, nk_list, nk_k_list,
        y, X, XTX, XTy, inv_V_β, inv_V_μ_β, label, J = 300)
    
    #preallocation and precomputation
    L_grid_deltasq = length(deltasq_grid);
    out_put = Array{Float64, 2}(undef, nk_k_list[k], L_grid_deltasq);
    #chol_inv_M = Array{Float64, 2}(undef, nk_list[k] + p, nk_list[k] + p);
    #u = Array{Float64, 1}(undef, nk_list[k] + p);
    
    
    invR_nk = compute_invR_nk(coords[:, CV_ind_ls[k]], ν = nu_pick, ϕ = phi_pick);
    R_k_nk = compute_R_k_nk(coords[:, CV_ind_hold_ls[k]], 
        coords[:, CV_ind_ls[k]], ν = nu_pick, ϕ = phi_pick);
    
    for i2 in 1:L_grid_deltasq
        deltasq_pick = deltasq_grid[i2];
        if p == 0
            chol_inv_M = cholesky(invR_nk + Diagonal(repeat([1 / deltasq_pick], nk_list[k]))); 
            u = y[CV_ind_ls[k]] /  deltasq_pick;
        else
            chol_inv_M = cholesky(Symmetric(vcat(
                        hcat(XTX_list[k] / deltasq_pick + inv_V_β, X[:, CV_ind_ls[k]] / deltasq_pick),
                        hcat(zeros(nk_list[k], p), 
                            invR_nk + Diagonal(repeat([1 / deltasq_pick], nk_list[k])))), :U));
            u = [inv_V_μ_β + XTy_list[k] / deltasq_pick; y[CV_ind_ls[k]] /  deltasq_pick];
        end

        ldiv!(chol_inv_M.U', u);  # u2 = chol_inv_M.L \ u;

        if label == "LSE"
            ## Stacking based on expectation
            # compute expectation of response in fold k
            ldiv!(chol_inv_M.U, u); # compute the posterior expectation of β and z u3 = chol_inv_M.L' \ u2;

            if p == 0
                out_put[:, i2] = 
                    (R_k_nk * invR_nk) * u;
            else
                out_put[:, i2] = 
                    (X[:, CV_ind_hold_ls[k]]' * u[1:p]) + (R_k_nk * invR_nk) * u[(p + 1):end];
            end
        else
            ## Stacking based on log point-wise predictive density
            if p == 0
                b_star = bσ + 0.5 * (y_sq_sum_list[k] / deltasq_pick - norm(u)^2);
            else
                b_star = bσ + 0.5 * (y_sq_sum_list[k] / deltasq_pick + 
                    dot(μβ, inv_V_μ_β) - norm(u)^2);
            end
            a_star = aσ + nk_list[k] / 2;

            ## generate posterior samples ##
            σ2_sam = rand(InverseGamma(a_star, b_star), J);

            ## compute the expected response on unobserved locations ##
            γ_sam = rand(Normal(), (J, (nk_list[k] + p)));   # each row is a sample
            lmul!(Diagonal(sqrt.(σ2_sam)), γ_sam);
            add_to_row!(γ_sam, u);          
            rdiv!(γ_sam, chol_inv_M.U');            # γ_sam' = backsolve(chol_inv_M, gamma.sam)

            # M_r the matrix of log ratios (lp), first, store the expected responses on the held out fold
            if p == 0
                M_r = γ_sam * (invR_nk * R_k_nk');
            else
                M_r = γ_sam[:, (1:p)] * X[:, CV_ind_hold_ls[k]] + 
                      γ_sam[:, (p+1):end] * (invR_nk * R_k_nk');
            end

            # the matrix of log ratios (lp)
            minus_to_row!(M_r, y[CV_ind_hold_ls[k]]);
            lmul!(Diagonal(sqrt.(1 / (deltasq_pick * σ2_sam))), M_r);
            square_and_plus!(M_r, log(2*pi));
            add_to_column!(M_r, log.(deltasq_pick * σ2_sam));
            M_r .*= -0.5;
            out_put[:, i2] = log.(mean(exp.(M_r), dims = 1))[1,:];
        end          
    end
    return out_put
end

stacking_prediction (generic function with 3 methods)

In [151]:
function stacking_prediction_LP(coords, nu_pick, phi_pick, deltasq_grid, 
        L_grid_deltasq, k, CV_ind_ls, CV_ind_hold_ls, p, nk_list, nk_k_list,
        y, X, XTX, XTy, inv_V_β, inv_V_μ_β, J = 300)
    ## compute expected log predictive density of response in fold k ##

    #preallocation and precomputation
    L_grid_deltasq = length(deltasq_grid);
    out_put = Array{Float64, 2}(undef, nk_k_list[k], L_grid_deltasq);
    chol_inv_M = Array{Float64, 2}(undef, nk_list[k] + p, nk_list[k] + p);
    u = Array{Float64, 1}(undef, nk_list[k] + p);
    a_star = 0.0; b_star = 0.0;
    σ2_sam = Array{Float64, 1}(undef, J);
    γ_sam = Array{Float64, 2}(undef, J, nk_list[k] + p);
    M_r = Array{Float64, 2}(undef, J, nk_k_list[k]);
    M_r_invRR_store = Array{Float64, 2}(undef, nk_list[k], nk_k_list[k]);
    if p > 0
        M_r_Xβ_store = Array{Float64, 2}(undef, J, nk_k_list[k]);
    end
    
    invR_nk = compute_invR_nk(coords[:, CV_ind_ls[k]], ν = nu_pick, ϕ = phi_pick);
    R_k_nk = compute_R_k_nk(coords[:, CV_ind_hold_ls[k]], 
        coords[:, CV_ind_ls[k]], ν = nu_pick, ϕ = phi_pick);
    
    for i2 in 1:L_grid_deltasq
        deltasq_pick = deltasq_grid[i2];
        if p == 0
            chol_inv_M[:] = invR_nk; 
            plus_cI!(chol_inv_M, 1 / deltasq_pick, p);
            cholesky!(Symmetric(chol_inv_M, :U));
            u[:] = y[CV_ind_ls[k]] /  deltasq_pick;
        else
            chol_inv_M[1:p, 1:p] = XTX_list[k] / deltasq_pick + inv_V_β;
            chol_inv_M[1:p, (p+1):end] = X[:, CV_ind_ls[k]] / deltasq_pick;
            chol_inv_M[(p+1):end, (p+1):end] = invR_nk; 
            plus_cI!(chol_inv_M, 1 / deltasq_pick, p);
            cholesky!(Symmetric(chol_inv_M, :U));
            u[:] = [inv_V_μ_β + XTy_list[k] / deltasq_pick; y[CV_ind_ls[k]] /  deltasq_pick];
        end

        ldiv!(UpperTriangular(chol_inv_M)', u);  # u2 = chol_inv_M.L \ u;

        ## Stacking based on log point-wise predictive density
        if p == 0
            b_star = bσ + 0.5 * (y_sq_sum_list[k] / deltasq_pick - norm(u)^2);
        else
            b_star = bσ + 0.5 * (y_sq_sum_list[k] / deltasq_pick + 
                dot(μβ, inv_V_μ_β) - norm(u)^2);
        end
        a_star = aσ + nk_list[k] / 2;

        ## generate posterior samples ##
        rand!(InverseGamma(a_star, b_star), σ2_sam);

        ## compute the expected response on unobserved locations ##
        rand!(Normal(), γ_sam)   # each row is a sample
        ldiagmul!(sqrt.(σ2_sam), γ_sam);
        add_to_row!(γ_sam, u);          
        rdiv!(γ_sam, UpperTriangular(chol_inv_M)');            # γ_sam' = backsolve(chol_inv_M, gamma.sam)

        # M_r the matrix of log ratios (lp), first, store the expected responses on the held out fold
        if p == 0
            mul!(M_r_invRR_store, invR_nk, R_k_nk');
            mul!(M_r, γ_sam, M_r_invRR_store);
        else
            mul!(M_r_invRR_store, invR_nk, R_k_nk');
            mul!(M_r, γ_sam[:, (p+1):end], M_r_invRR_store);
            mul!(M_r_Xβ_store, γ_sam[:, (1:p)], X[:, CV_ind_hold_ls[k]]);
            M_r += M_r_Xβ_store;
        end

        # the matrix of log ratios (lp)
        minus_to_row!(M_r, y[CV_ind_hold_ls[k]]);
        ldiagmul!(sqrt.(1 ./ (deltasq_pick .* σ2_sam)), M_r);
        square_and_plus!(M_r, log(2*pi));
        add_to_column!(M_r, log.(deltasq_pick .* σ2_sam));
        M_r .*= -0.5;
        out_put[:, i2] = log.(mean(exp.(M_r), dims = 1))[1,:];
    end          
    return out_put
end

stacking_prediction_LP (generic function with 3 methods)

In [152]:
@benchmark stacking_prediction_LP(coords, nu_pick, phi_pick, deltasq_grid, 
                L_grid_deltasq, k, CV_ind_ls, CV_ind_hold_ls, p, 
                nk_list, nk_k_list, y, X, XTX, XTy, inv_V_β, inv_V_μ_β, J)

BenchmarkTools.Trial: 111 samples with 1 evaluation.
 Range (min … max):  43.733 ms … 49.662 ms  ┊ GC (min … max): 0.00% … 5.25%
 Time  (median):     44.509 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   45.056 ms ±  1.372 ms  ┊ GC (mean ± σ):  1.06% ± 2.10%

  ▂█  ▂ ▂▄▂                                                    
  ██▆▆█████▇▆▇▃▃▃▃▁▃▃▁▁▁▁▁▃▁▁▃▄▄▆▃▃▅▆▃▃▁▃▁▃▁▃▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▃ ▃
  43.7 ms         Histogram: frequency by time        49.6 ms <

 Memory estimate: 8.62 MiB, allocs estimate: 79319.

In [46]:
## improve efficiency ##
prog = Progress(size(grid_phi_nu, 1), 1, "Computing initial pass...", 50)
for i1 in 1:size(grid_phi_nu, 1)
    phi_pick = grid_phi_nu[i1, 1];
    nu_pick = grid_phi_nu[i1, 2];
    for k in 1:K_fold
        if label == "LSE"
            y_expect[CV_ind_hold_ls[k], 
                (i1 - 1) * L_grid_deltasq .+ (1:L_grid_deltasq)] = 
            stacking_prediction_LSE(coords, nu_pick, phi_pick, deltasq_grid, 
                L_grid_deltasq, k, CV_ind_ls, CV_ind_hold_ls, p, 
                nk_list, y, X, XTX, XTy, inv_V_β, inv_V_μ_β);
        else
            lp_expect[CV_ind_hold_ls[k], 
                (i1 - 1) * L_grid_deltasq .+ (1:L_grid_deltasq)] = 
            stacking_prediction_LP(coords, nu_pick, phi_pick, deltasq_grid, 
                L_grid_deltasq, k, CV_ind_ls, CV_ind_hold_ls, p, 
                nk_list, y, X, XTX, XTy, inv_V_β, inv_V_μ_β, J);
        end
            
    end
    next!(prog)
end

Computing initial pass...100%|██████████████████████████████████████████████████| Time: 0:00:06
